<a href="https://colab.research.google.com/github/raphaelp-silva/deep_learning_com_pytorch_e_python/blob/main/Tarefa_Tuning_regress%C3%A3o_carros_usados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa: Tuning regressão carros usados

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 3.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import torch.nn as nn
from skorch import NeuralNetRegressor
import torch
from sklearn.model_selection import GridSearchCV
torch.__version__

'2.5.1+cu124'

## Etapa 2: Base de dados

In [3]:
torch.manual_seed(123)

In [4]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

base = base[base.price > 10]
base = base.loc[base.price < 350000]

valores = {'vehicleType': 'limousine', 'gearbox': 'manuell',
           'model': 'golf', 'fuelType': 'benzin',
           'notRepairedDamage': 'nein'}
base = base.fillna(value = valores)

previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values.reshape(-1, 1)

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
onehotencoder = ColumnTransformer(transformers = [("OneHot", OneHotEncoder(), [0, 1, 3, 5, 8, 9, 10])], remainder = 'passthrough')
previsores = onehotencoder.fit_transform(previsores).toarray()

previsores = previsores.astype('float32')
preco_real = preco_real.astype('float32')

In [5]:
previsores.shape

(269609, 317)

## Etapa 3: Construção do modelo

In [11]:
class regressor_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(317, 158)
        torch.nn.init.uniform_(self.dense0.weight)
        self.dense1 = nn.Linear(158, 158)
        torch.nn.init.uniform_(self.dense1.weight)
        self.dense2 = nn.Linear(158, 1)

        self.activation = nn.ReLU()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dense2(X)
        return X

In [12]:
regressor_sklearn = NeuralNetRegressor(module = regressor_torch,
                                       optimizer = torch.optim.Adam,
                                       max_epochs = 100,
                                       batch_size = 300,
                                       train_split = False)

## Etapa 4: Tuning dos parâmetros

In [13]:
params = {'criterion': [torch.nn.MSELoss, torch.nn.L1Loss, torch.nn.SmoothL1Loss]}

In [14]:
grid_search = GridSearchCV(estimator = regressor_sklearn,
                           param_grid = params,
                           cv = 5)

In [ ]:
grid_search = grid_search.fit(previsores, preco_real)

  epoch         train_loss     dur
-------  -----------------  ------
      1  409746253339.3449  4.2352
      2  97220736.6100  4.2442
      3  96022920.0120  3.6240
      4  95797013.7261  4.0008
      5  1199774627.6839  3.6426
      6  2355691602.5791  3.5399
      7  1480782707.1292  3.6944
      8  1019189690.7948  3.7469
      9  427489208.0012  3.5913
     10  356636783.3693  3.8187


In [ ]:
melhores_parametros = grid_search.best_params_
melhor_resultado = grid_search.best_score_